In [2]:
!pip3 install beautifulsoup4 
!pip3 install requests
!pip3 install selenium
!pip3 install webdriver-manager

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no lon

In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd

In [231]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

def scrape_scroll_articles(link, tag, class_name, scroll_num, sleep_time):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(link)

    for i in range(1,scroll_num):
        driver.execute_script("window.scrollTo(1,50000)")
        time.sleep(sleep_time)

    file = open('DS.html', 'w')
    file.write(driver.page_source)
    file.close()

    driver.close()

    data = open('./DS.html','r')
    soup = BeautifulSoup(data,"html.parser")
    results = soup.find_all(tag, {'class': class_name}) if class_name != None else soup.find_all(tag)
    
    urls = []
    if tag != "div":
        for result in results:
            link = result["href"]
            urls.append(link)
    else:
        for result in results:
            link = result.find("a")["href"]
            urls.append(link)
    
    return urls
    

In [8]:
def scrape_article_aljazirah(links, section):
    content = []
    title = []
    author = []
    urls = []
    sec = []
    for url in links:
        res = get(url)
        html_soup = BeautifulSoup(res.text, 'html.parser')
        article_containers = html_soup.find_all('p')
        article_title = html_soup.find('h1', {"class": "entry-title title flipboard-title"})
        article_author = article_containers[2].get_text() if article_containers[2] != None else "N/A"
        article_containers = article_containers[3:-2] if article_containers[3:-2] != None else "N/A"
        article_title = article_title.get_text() if article_title != None else "N/A"
        urls.append(url)
        author.append(article_author)
        content.append(" ".join([data.get_text() for data in article_containers]))
        title.append(article_title)
        sec.append(section)
    return {"url": urls, "title": title, 
		"author": author, "content": content, "section": sec}

In [ ]:
import requests
links = {
    "local":"https://www.al-jazirahonline.com/category/%d8%a7%d9%84%d9%85%d9%85%d9%84%d9%83%d8%a9/",
    "world":"https://www.al-jazirahonline.com/category/world/",
    "sport":"https://www.al-jazirahonline.com/category/sport/",
    "women":"https://www.al-jazirahonline.com/category/%d8%a7%d9%84%d9%85%d8%b1%d8%a3%d8%a9/",
    "culture":"https://www.al-jazirahonline.com/category/%d8%a7%d9%84%d9%85%d8%ac%d8%aa%d9%85%d8%b9/"
}
data = {"url": [], "title": [], 
		"author": [], "content": [], "section": []}
for section, link in links.items():
    print(section)
    articles_urls = []
    for i in range(1, 200):
        if i == 1:
            response = requests.get(link)
        else:
              response = requests.get(f"{link}page/{i}")
        aljazirah_soup = BeautifulSoup(response.text, "html.parser")
        page_links = aljazirah_soup.find_all("a", {"class": "mask-img"})
        for url in page_links:
            articles_urls.append(url["href"])
    section_articles = scrape_article_aljazirah(articles_urls, section)
    data["url"].extend(section_articles["url"])
    data["title"].extend(section_articles["title"])
    data["author"].extend(section_articles["author"])
    data["content"].extend(section_articles["content"])
    data["section"].extend(section_articles["section"])

In [150]:
df = pd.DataFrame(data=data)

df.to_csv("al-jazirah.csv" ,index=False)

In [143]:
len(data['content']), len(data['author']), len(data['title'])

(1116, 1116, 1116)

In [258]:
from requests import get

def scrape_article_okaz(links):
    content = []
    title = []
    author = []
    url = []
    sec = []
    for section, section_urls in links.items():
        print(section)
        for url in section_urls:
            res = get(url)
            html_soup = BeautifulSoup(res.text, 'html.parser')
            article_containers = html_soup.find('div', {"class": "bodyText"})
            title = html_soup.find('h1')
            has_child_nodes = len(article_containers.find_all("p")) != 0
            author = html_soup.find("ul", {"class": "list-meta-items header-top show-for-desktop"})
            url.append(url)
            author.append(author.find_all("li")[1].get_text())
            if has_child_nodes: 
                content.append(" ".join([data.get_text() for data in article_containers]))
            else:
                content.append(article_containers.get_text())
            title.append(title.get_text())
            if section == "news":
                sec.append(url.split('/')[4])
            else:
                sec.append(section)
    return {"url": url, "title": title, 
		"author": author, "content": content, "section": sec}

In [264]:
headers = {'User-Agent': 'Mozilla/5.0'}


session = requests.Session()
sections = {190: "news", 3: "sport", 4: "economy", 8: "culture"}
    
articles_urls = {"news": [], "sport": [], "economy": [], "culture": []}

for section in sections.keys():
    for i in range(1, 200):
        payload = {"page":i, "section":section, "sub_section":0}
        response = session.post('https://www.okaz.com.sa/ajax-more-articles',headers=headers,data=payload)
        if response.headers["content-type"].strip().startswith("application/json"):
            soup = BeautifulSoup(response.json()["html"],"html.parser")
        else: soup = BeautifulSoup(response.text,"html.parser")
        results = soup.find_all("div", {'class': "update"}) 
        
        for result in results:
            link = result.find("a")["href"]
            articles_urls[sections[section]].append(link)

In [265]:
data = {"url": [], "title": [], 
		"author": [], "content": [], "section": []}

section_articles = scrape_article_okaz(articles_urls)
data["url"].extend(section_articles["url"])
data["title"].extend(section_articles["title"])
data["author"].extend(section_articles["author"])
data["content"].extend(section_articles["content"])
data["section"].extend(section_articles["section"])


news
sport
economy
culture


In [269]:
df = pd.DataFrame(data=data)

In [270]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13515 entries, 0 to 13514
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      13515 non-null  object
 1   title    13515 non-null  object
 2   author   13515 non-null  object
 3   content  13515 non-null  object
 4   section  13515 non-null  object
dtypes: object(5)
memory usage: 528.1+ KB


In [201]:
df = pd.DataFrame(data=data)

df.to_csv("okaz.csv" ,index=False)

In [217]:
from requests import get

def scrape_article_sabq(links, section):
    sabq_content = []
    sabq_title = []
    sabq_author = []
    sabq_url = []
    sabq_sec = []
    for url in links:
        res = get(url)
        html_soup = BeautifulSoup(res.text, 'html.parser')
        article_containers = html_soup.find('div', {"class": "arr--story-page-card-wrapper"})
        title = html_soup.find('h1')
        author = html_soup.find("h5")
        author = author.get_text() if author != None else "N/A"
        article_containers = article_containers.get_text() if article_containers != None else "N/A"
        title = title.get_text() if title != None else "N/A"
        sabq_url.append(url)
        sabq_author.append(author)
        sabq_content.append(article_containers)
        sabq_title.append(title)
        sabq_sec.append(section)
    return {"url": sabq_url, "title": sabq_title, 
		"author": sabq_author, "content": sabq_content, "section": sabq_sec}

In [218]:
import requests

sections = ["sports", "saudia", "world", "mylife", "stations", "tourism", "technology", "cars", "business"]
data = {"url": [], "title": [], 
		"author": [], "content": [], "section": []}
for section in sections:
    response = requests.get("https://sabq.org/api/v1/collections/" + section + "?item-type=story&limit=1000") 
    links = []
    print(section)
    for item in response.json()["items"]:
        links.append(item["story"]["url"])
    section_articles = scrape_article_sabq(links, section)
    data["url"].extend(section_articles["url"])
    data["title"].extend(section_articles["title"])
    data["author"].extend(section_articles["author"])
    data["content"].extend(section_articles["content"])
    data["section"].extend(section_articles["section"])

sports
saudia
world
mylife
stations
tourism
technology
cars
business


In [219]:
df = pd.DataFrame(data=data)

In [220]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8349 entries, 0 to 8348
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      8349 non-null   object
 1   title    8349 non-null   object
 2   author   8349 non-null   object
 3   content  8349 non-null   object
 4   section  8349 non-null   object
dtypes: object(5)
memory usage: 326.3+ KB


In [223]:
df.head(20)

,url,title,author,content,section
0,https://sabq.org/sports/vtx4t2tf2w,الأولمبي الآسيوي يرحب بطلب المملكة استضافة دور...,سبق,رحب المجلس الأولمبي الآسيوي ومقره دولة الكويت ...,sports
1,https://sabq.org/sports/9jr7prlkn4,"""النصر"" و""الاتفاق"" مهتمان بالمدافع البلجيكى ""ج...",محمد السيد,"كشف موقع ""footmercato"" الفرنسي أن البلجيكي جيس...",sports
2,https://sabq.org/sports/ivh1nowgtl,ضمن معسكره في مدينة بوردور .. الأخضر الأولمبي ...,سبق,أجرى المنتخب السعودي تحت 23 عامًا لكرة القدم ،...,sports
3,https://sabq.org/sports/5kp7sl0x1w,"كان ضمن اهتمامات ""الاتحاد"" .. ""بن شرقي"" يقترب ...",محمد السيد,كشفت تقارير إعلامية أن المحترف المغربي أشرف بن...,sports
4,https://sabq.org/sports/lyvfk5u786,صربيا تجهز حكام دوري كأس الأمير محمد بن سلمان ...,سبق,دشن حكام دوري كأس الأمير محمد بن سلمان للمحترف...,sports
5,https://sabq.org/sports/8q5gscb7ma,بعد ودية كوكسي .. الاتفاق يستأنف تدريباته وباخ...,سبق,وسط أجواء كثيفة من الضباب استأنف الفريق الأول ...,sports
6,https://sabq.org/sports/du56fjofza,قمة الجولة العاشرة تجمع الهلال والنصر.. في ملع...,أحمد سرور,اعتمدت لجنة المسابقات في الرابطة جدول مباريات ...,sports
7,https://sabq.org/sports/75d0jgjvr5,انطلاق معسكر برنامج الابتعاث السعودي لتطوير مو...,سبق,انطلق اليوم معسكر برنامج الابتعاث السعودي لتط...,sports
8,https://sabq.org/sports/3yth9rvpv9,"قادمًا من الأهلي.. ""السومة"" ينتقل للدوري القطري",سبق,ظفر نادي العربي القطري بخدمات مهاجم الفريق الك...,sports
9,https://sabq.org/sports/7nvhbxuql2,"بنظام الإعارة من النصر.. ""قاسم"" يقترب من التوق...",سبق,أشارت تقارير محلية، اليوم الخميس، إلى أن ظهير ...,sports


In [224]:
df.to_csv("sabq.csv" ,index=False)

In [290]:
def scrape_article_alriyadh(links, section):
    content = []
    title = []
    author = []
    url = []
    sec = []
    for url in links:
        res = get(url, verify=False)
        html_soup = BeautifulSoup(res.text, 'html.parser')
        article_containers = html_soup.find('div', {"class": "col-md-12 article-text"}).get_text()
        title = html_soup.find('h2').get_text()
        author = html_soup.find("div", {"class": "col-md-12 col-lg-3"}).get_text()
        url.append(url)
        author.append(author)
        content.append(article_containers)
        title.append(title)
        sec.append(section)
    return {"url": url, "title": title, 
		"author": author, "content": content, "section": sec}

In [307]:
import warnings
warnings.filterwarnings("ignore")

sections = {"sports": "/news.sport", "local": "/news.local", "world": "/news.inter", "economy": "/news.econ", "culture": "/culture"}
data = {"url": [], "title": [], 
		"author": [], "content": [], "section": []}
for section, url in sections.items():
    print(section)
    links = []
    for i in range(1, 1000):
        response = requests.get("https://www.alriyadh.com" + url + "?&page=" + str(i), verify=False) 
        riyadh_soup = BeautifulSoup(response.text, "html.parser")
        page_links = riyadh_soup.find_all("h4")
        for link in page_links:
            links.append("https://www.alriyadh.com" + link.find("a")["href"])
    section_articles = scrape_article_alriyadh(links, section)
    data["url"].extend(section_articles["url"])
    data["title"].extend(section_articles["title"])
    data["author"].extend(section_articles["author"])
    data["content"].extend(section_articles["content"])
    data["section"].extend(section_articles["section"])

sports
local
world
economy
culture


In [308]:
df = pd.DataFrame(data=data)

In [310]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260 entries, 0 to 2259
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2260 non-null   object
 1   title    2260 non-null   object
 2   author   2260 non-null   object
 3   content  2260 non-null   object
 4   section  2260 non-null   object
dtypes: object(5)
memory usage: 88.4+ KB


In [311]:
df.to_csv("alriyadh.csv", index=False)

In [313]:
okaz = pd.read_csv("okaz.csv", lineterminator='\n')
aljazirah = pd.read_csv("al-jazirah.csv")
sabq = pd.read_csv("sabq.csv")
alriyadh = pd.read_csv("alriyadh.csv")

In [314]:
okaz.shape

(13515, 5)

In [316]:
sabq.shape

(8350, 5)

In [318]:
aljazirah.shape

(1116, 5)

In [319]:
alriyadh.shape

(2260, 5)